In [2]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import os

# 读取shapefile
shapefile = "shp_path.shp"
gdf = gpd.read_file(shapefile)

# 读取大图tif文件
tif_file = "tif_path.tif"

# 设置输出文件夹
output_folder = "output_dir_path"
os.makedirs(output_folder, exist_ok=True)

with rasterio.open(tif_file) as src:
    for idx, row in gdf.iterrows():
        # 获取框的几何信息
        geometry = [row['geometry']]
        
        # 跳过无效的几何形状
        if geometry[0] is None:
            continue
        
        # 裁剪大图
        try:
            out_image, out_transform = mask(src, geometry, crop=True)
        except Exception as e:
            print(f"裁剪第 {idx} 个框时出错: {e}")
            continue
        
        # 创建子文件夹
        sub_folder = os.path.join(output_folder, f"output_{idx}")
        os.makedirs(sub_folder, exist_ok=True)
        
        # 保存裁剪后的图像
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        
        output_tif = os.path.join(sub_folder, f"output_image_{idx}.tif")
        with rasterio.open(output_tif, "w", **out_meta) as dest:
            dest.write(out_image)
        
        # 为裁剪后的图像生成新的shapefile
        new_gdf = gpd.GeoDataFrame({'geometry': geometry}, crs=gdf.crs)
        output_shapefile = os.path.join(sub_folder, f"output_shape_{idx}.shp")
        new_gdf.to_file(output_shapefile)

print("裁剪和shapefile生成完成")

E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attri

裁剪和shapefile生成完成


In [11]:
##上面的代码对大图进行裁剪输出小图tif与shp文件，其中tif图像的倾角发生变换，
##并且部分图形从矩形变成了平行四边形，但不是裁剪的shp文件的问题
##因为我运行拼接shp的代码输出的merge.shp与ArcGIS标注输出的output.shp文件打开是完全一致的
##下面的代码在裁剪时使用的外接矩形来进行裁剪，不存在变换角度与形状的问题，但标注的图像范围稍大了一点。

In [7]:
import os
import fiona
import rasterio
from rasterio.mask import mask
from shapely.geometry import shape, mapping, Polygon
from shapely.affinity import rotate
import numpy as np

def get_minimum_rotated_rectangle(geom):
    """获取最小外接矩形并旋转至水平"""
    min_rect = geom.minimum_rotated_rectangle
    coords = list(min_rect.exterior.coords)
    edge1 = np.array(coords[1]) - np.array(coords[0])
    edge2 = np.array(coords[2]) - np.array(coords[1])
    edge = edge1 if np.linalg.norm(edge1) > np.linalg.norm(edge2) else edge2
    angle = np.degrees(np.arctan2(edge[1], edge[0]))
    rotated_rect = rotate(min_rect, -angle, origin='center')
    return rotated_rect

def crop_and_save_raster(input_tif, input_shp, output_dir):
    # 创建输出目录
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 读取shp文件
    with fiona.open(input_shp, 'r') as shp:
        features = [feature for feature in shp]
        schema = shp.schema
        crs = shp.crs
    
    # 读取大tif图像
    with rasterio.open(input_tif) as src:
        for i, feature in enumerate(features):
            try:
                # 获取标注框的几何形状
                geom = shape(feature['geometry'])
                
                # 如果几何对象为空，则跳过
                if geom.is_empty:
                    continue
                
                # 计算最小外接矩形并旋转至水平
                rotated_rect = get_minimum_rotated_rectangle(geom)
                
                # 裁剪tif图像
                out_image, out_transform = mask(src, [mapping(rotated_rect)], crop=True)
                
                # 创建子文件夹
                subfolder = os.path.join(output_dir, f'crop_{i}')
                if not os.path.exists(subfolder):
                    os.makedirs(subfolder)
                
                # 输出文件路径
                output_tif = os.path.join(subfolder, f'crop_{i}.tif')
                output_shp = os.path.join(subfolder, f'crop_{i}.shp')
                
                # 保存裁剪后的tif图像
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
                })
                
                with rasterio.open(output_tif, 'w', **out_meta) as dest:
                    dest.write(out_image)
                
                # 创建新的shp文件
                with fiona.open(output_shp, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as dest_shp:
                    new_feature = {
                        'geometry': mapping(rotated_rect),
                        'properties': feature['properties']
                    }
                    dest_shp.write(new_feature)
            
            except Exception as e:
                print(f"Error processing feature {i}: {e}")

# 输入文件路径
input_tif = 'G:\\标注图\\4.1标注图\\标注4\\result.tif'
input_shp = 'G:\\标注图\\4.1标注图\\标注4\\2_Output.shp'
output_dir = 'G:\\shp_kuang4pro'

# 执行裁剪并保存
crop_and_save_raster(input_tif, input_shp, output_dir)


E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attri

Error processing feature 9: 'NoneType' object has no attribute 'get'
Error processing feature 10: 'NoneType' object has no attribute 'get'


E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attri

Error processing feature 20: 'NoneType' object has no attribute 'get'


E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attribute is shadowing the alpha band. All masks will be determined by the nodata attribute
  out_image = dataset.read(
E:\Anaconda3\envs\CV\lib\site-packages\rasterio\mask.py:189: NodataShadowWarning: The dataset's nodata attri